# Big Shadow State Tomography

In [1]:
import numpy as np

from time import time
from tqdm import tqdm

import qiskit

from qiskit_aer import AerSimulator
from qiskit.visualization import plot_gate_map 

from qiskit_ibm_runtime.fake_provider import FakeLimaV2
from qiskit_ibm_runtime.fake_provider import FakeMontrealV2
from qiskit_ibm_runtime.fake_provider import FakeKyiv

from qiskit.circuit.random import random_circuit

import matplotlib.pyplot as plt

plt.style.use("dark_background")

%cd -q ../..

from rivet_transpiler import transpile_right

In [2]:
FAKE_BACKEND = FakeKyiv()

backend = AerSimulator.from_backend(FAKE_BACKEND)

backend.options.noise_model = None

# qiskit.visualization.plot_gate_map(FAKE_BACKEND)

#### 3.3. Functions

In [3]:
def rotate_qubit_to_basis(circuit, qubit, basis):

    if basis == "X":
        circuit.h(qubit)

    elif basis == "Y":
        circuit.sdg(qubit)
        circuit.h(qubit)

    elif basis == "Z":
        pass

In [4]:
def build_bases_rotation_circuit(bases):

    qubits_count = len(bases)

    circuit = qiskit.QuantumCircuit(qubits_count)

    for qubit, basis in enumerate(bases):

        if basis == "X":
            circuit.h(qubit)

        elif basis == "Y":
            circuit.sdg(qubit)
            circuit.h(qubit)

        elif basis == "Z":
            pass

    return circuit

In [5]:
def get_inverted_channel(channel, qubits_count):

    identity = np.eye(2 ** qubits_count)

    inverted_channel = (2 ** qubits_count + 1) * channel - identity

    return inverted_channel

In [6]:
def get_basis_unitary(basis):

    if basis == "X":
        unitary = 1/np.sqrt(2) * np.array([[1.,1.],
                                           [1.,-1.]])
    elif basis == "Y":
        unitary = 1/np.sqrt(2) * np.array([[1.,-1.0j],
                                           [1.,1.j]])
    elif basis == "Z":
        unitary = np.eye(2)

    return unitary

### Circuit

In [37]:
# Parameters

# QUBITS_COUNT = 4
QUBITS_COUNT = 127
CIRCUIT_DEPTH = 100

OPTIMIZATION_LEVEL = 3

SEED = 1234

rng = np.random.default_rng(SEED) 

qubits = list(range(QUBITS_COUNT))

## Random Circuit

circuit = random_circuit(
    QUBITS_COUNT, 
    depth=CIRCUIT_DEPTH,
    seed=SEED,
    max_operands=1, 
    measure=False)

# circuit.draw(fold=-1)

### 4.2.  Run Paulis Experiment

In [38]:
TOTAL_SHOTS_COUNT = 10

In [39]:
random_bases = rng.choice(['X','Y','Z'], size=(TOTAL_SHOTS_COUNT, QUBITS_COUNT))

bases_list, shots_counts = np.unique(random_bases, axis=0, return_counts=True)

bases_list.shape

(10, 127)

In [40]:
rotation_circuits = []

for bases in bases_list:

    rotation_circuit = build_bases_rotation_circuit(bases)

    rotation_circuits.append(rotation_circuit)

len(rotation_circuits)

10

#### Basic Transpilation - Slower Version!


In [41]:
start_time = time()

# Compose and Transpile all Rotation Circuits

shadow_circuits = []

rotation_tracker = tqdm(rotation_circuits, unit=' basis', ncols=110)

for rotation_circuit in rotation_tracker:

    measurement_circuit = rotation_circuit.measure_all(inplace=False)

    measured_circuit = circuit.compose(measurement_circuit)

    # display(measured_circuit.draw(fold=-1))

    shadow_circuit = qiskit.transpile(
        measured_circuit,
        backend,
        seed_transpiler=SEED,
        optimization_level=OPTIMIZATION_LEVEL)

    shadow_circuits.append(shadow_circuit)

estimated_time = time() - start_time

print(f"estimated_time: {estimated_time:.02f}")

100%|█████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.98s/ basis]

estimated_time: 29.83


#### Transpile Right - Much Faster!

In [42]:
start_time = time()

# Transpile Central Part

transpiled_central_circuit = qiskit.transpile(
    circuit,
    backend,
    seed_transpiler=SEED,
    optimization_level=OPTIMIZATION_LEVEL)


# Transpile Right all Rotation Circuits

shadow_circuits = []

rotation_tracker = tqdm(rotation_circuits, unit=' basis', ncols=110)

for rotation_circuit in rotation_tracker:

    measurement_circuit = rotation_circuit.measure_all(inplace=False)

    shadow_circuit = transpile_right(
        transpiled_central_circuit,
        measurement_circuit,
        backend=backend,
        seed_transpiler=SEED,
        optimization_level=OPTIMIZATION_LEVEL)

    shadow_circuits.append(shadow_circuit)


estimated_time = time() - start_time

print(f"estimated_time: {estimated_time:.02f}")

100%|█████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.72 basis/s]

estimated_time: 4.16
